<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3ef27cc361c82de59b1a1e3abf8ae9aa4f7eea78e48283f4506bbbe68ade6ad6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-01 11:48:31
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -25.63 K (-0.18%)
Current PnL: -22.91 L (-15.01%)
CY Booked + Current PnL: -9.02 L (-5.91%)
-------------------
Total profit:  1.60 L
Total loss:  -24.52 L
-------------------
Total Booked + Current PnL: 17.96 L (14.3%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.72%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.32 L (61.81%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.32,-8.53,9.34,0.02,14914.0,-14884.0,159675.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.89,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.22,-16.51,19.88,0.09,16727.0,-16636.0,84141.0,95.30,37.0,M-SC,3.22,253.0,-0.99,0.59,10.54,OX40N,NTT,DURABLES
43,ITC,452.00,-0.19,-1.41,12.02,10.44,23717.0,-2826.0,197312.0,-41.15,45.0,X-LC,1.32,5.0,-0.12,1.38,4.07,X40,NTT,FMCG
83,VOLTAS,1530.00,-0.90,6.70,12.18,19.69,24918.0,12843.0,204585.0,-3.70,47.0,X-MC,3.23,78.0,0.52,1.43,13.95,XY25,NTT,AC
51,MASFIN,398.61,1.44,-4.09,27.25,22.05,25608.0,-4005.0,93975.0,-17.34,54.0,H-SC,6.87,164.0,-0.16,0.66,36.60,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,4.77,-0.96,50.07,48.63,99152.0,-1927.0,198027.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.48,XR,NTT,MISC
81,VALIANTORG,838.00,4.54,-45.21,217.85,74.15,195283.0,-73964.0,89641.0,-260.95,37.0,H-SC,18.17,149.0,-0.38,0.63,14.91,XR,NTT,CHEMICALS
15,CAMS,4762.00,2.03,3.49,20.59,24.79,53665.0,8778.0,260634.0,-1.25,55.0,X-SC,1.89,86.0,0.16,1.82,27.43,X40N,NTT,MISC
51,MASFIN,398.61,1.44,-4.09,27.25,22.05,25608.0,-4005.0,93975.0,-17.34,54.0,H-SC,6.87,164.0,-0.16,0.66,36.60,XR,ATH,FINANCE
70,SURYODAY,216.00,1.42,-18.36,50.71,23.03,74131.0,-32885.0,146186.0,60.09,51.0,H-SC,9.72,167.0,-0.44,1.02,45.05,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-5.61,-18.90,124.31,81.92,115738.0,-21694.0,93104.0,-52.45,21.0,M-SC,2.92,236.0,-0.19,0.65,10.22,XR,NTT,DURABLES
27,GLAXO,3466.20,-2.06,1.02,37.74,39.15,75978.0,2036.0,201320.0,-18.92,43.0,X-MC,10.52,60.0,0.03,1.41,30.32,X40,ATH,PHARMA
28,GREENPANEL,537.00,-1.65,-34.67,119.09,43.14,142442.0,-63469.0,119609.0,168.09,28.0,M-SC,6.32,240.0,-0.45,0.84,12.51,XY24,NTT,MISC
58,RAJOOENG,143.10,-1.62,-28.83,96.16,39.61,70149.0,-29550.0,72950.0,-51.80,33.0,H-SC,21.87,135.0,-0.42,0.51,2.82,AR,ATH,MISC
54,PAGEIND,51605.07,-1.53,-7.82,36.76,26.07,55486.0,-12800.0,150940.0,-32.16,30.0,X-MC,9.90,55.0,-0.23,1.06,0.00,X40,ATH,APPARELS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.42,-0.31,107.37,106.72,132010.0,-387.0,122949.0,-50.63,38.0,H-SC,2.38,139.0,-0.00,0.86,22.99,AR,ATH,MISC
37,IEX,219.00,4.77,-0.96,50.07,48.63,99152.0,-1927.0,198027.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.48,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.45,-4.69,118.15,107.91,166258.0,-6926.0,140718.0,-22.58,37.0,M-SC,10.83,216.0,-0.04,0.98,0.62,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.32,-8.53,9.34,0.02,14914.0,-14884.0,159675.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.89,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.22,-16.51,19.88,0.09,16727.0,-16636.0,84141.0,95.30,37.0,M-SC,3.22,253.0,-0.99,0.59,10.54,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.98,-21.84,45.19,13.47,95243.0,-58905.0,210762.0,-67.92,36.0,H-SC,1.75,158.0,-0.62,1.48,7.17,XY24,NTT,PAINTS
67,SIS,528.00,-0.82,-25.00,61.69,21.26,51369.0,-27762.0,83270.0,1963.21,43.0,H-SC,3.71,166.0,-0.54,0.58,12.51,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.06,2.28,68.42,72.27,118585.0,3871.0,173319.0,-12.43,59.0,M-LC,3.51,99.0,0.03,1.21,8.6,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.06,2.28,68.42,72.27,118585.0,3871.0,173319.0,-12.43,59.0,M-LC,3.51,99.0,0.03,1.21,8.60,XR,NTT,IT
38,INDIAMART,4810.62,0.42,-0.31,107.37,106.72,132010.0,-387.0,122949.0,-50.63,38.0,H-SC,2.38,139.0,-0.00,0.86,22.99,AR,ATH,MISC
37,IEX,219.00,4.77,-0.96,50.07,48.63,99152.0,-1927.0,198027.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.48,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.87,-1.48,37.66,35.63,77760.0,-3094.0,206480.0,-15.41,41.0,H-MC,3.71,119.0,-0.04,1.45,14.95,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.45,-4.69,118.15,107.91,166258.0,-6926.0,140718.0,-22.58,37.0,M-SC,10.83,216.0,-0.04,0.98,0.62,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.44,-36.47,112.82,35.20,92222.0,-46927.0,81743.0,7.15,21.0,X-SC,13.64,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.72,-14.68,74.00,48.45,126590.0,-29438.0,171068.0,-20.73,27.0,X-MC,7.78,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
46,JSWINFRA,342.00,-0.00,-8.91,26.50,15.23,48359.0,-17861.0,182486.0,-22.11,29.0,X-MC,6.90,66.0,-0.37,1.28,20.46,X40N,NTT,REALTY
54,PAGEIND,51605.07,-1.53,-7.82,36.76,26.07,55486.0,-12800.0,150940.0,-32.16,30.0,X-MC,9.90,55.0,-0.23,1.06,0.00,X40,ATH,APPARELS
76,TMPV,600.00,1.19,-14.44,66.18,42.18,156508.0,-39922.0,236488.0,-23.08,33.0,X-LC,2.32,3.0,-0.26,1.66,2.44,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.19,-1.41,12.02,10.44,23717.0,-2826.0,197312.0,-41.15,45.0,X-LC,1.32,5.0,-0.12,1.38,4.07,X40,NTT,FMCG
3,ACC,3906.0,0.08,-22.17,110.91,64.15,205405.0,-52751.0,185200.0,-54.57,49.0,X-SC,1.57,82.0,-0.26,1.30,3.89,XY24,BTT,CEMENT
15,CAMS,4762.0,2.03,3.49,20.59,24.79,53665.0,8778.0,260634.0,-1.25,55.0,X-SC,1.89,86.0,0.16,1.82,27.43,X40N,NTT,MISC
66,SIEMENS,4671.5,-0.61,-11.96,42.56,25.51,69730.0,-22255.0,163840.0,0.51,61.0,H-LC,1.99,49.0,-0.32,1.15,20.74,AR,ATH,ELECTRICAL
36,ICICIPRULI,790.0,0.09,2.83,27.36,30.97,49726.0,5010.0,181748.0,-21.38,55.0,X-MC,2.12,75.0,0.10,1.27,15.75,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.44,-36.47,112.82,35.20,92222.0,-46927.0,81743.0,7.15,21.0,X-SC,13.64,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.72,-14.68,74.00,48.45,126590.0,-29438.0,171068.0,-20.73,27.0,X-MC,7.78,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-1.53,-7.82,36.76,26.07,55486.0,-12800.0,150940.0,-32.16,30.0,X-MC,9.90,55.0,-0.23,1.06,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,0.38,-4.36,39.86,33.76,76555.0,-8760.0,192060.0,-32.51,35.0,X-MC,4.06,57.0,-0.11,1.34,0.61,X40,ATH,FMCG
19,COLPAL,3726.84,-0.47,-18.05,72.67,41.51,156844.0,-47535.0,215830.0,-2.34,41.0,X-MC,7.21,61.0,-0.30,1.51,1.28,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.38,-27.77,97.77,42.84,45904.0,-18055.0,46951.0,-52.62,42.0,X-SC,37.51,83.0,-0.39,0.33,2.16,XY24,NTT,MISC
60,RELAXO,1176.00,-0.15,-47.08,187.78,52.31,144264.0,-68334.0,76826.0,-43.66,37.0,X-SC,4.81,91.0,-0.47,0.54,2.84,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.44,-36.47,112.82,35.20,92222.0,-46927.0,81743.0,7.15,21.0,X-SC,13.64,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.72,-6.90,30.60,21.58,39922.0,-9675.0,130464.0,-53.04,34.0,X-MC,6.24,56.0,-0.24,0.91,15.50,X40N,ATH,IT
52,MEDANTA,1486.00,-0.65,3.36,20.04,24.08,26296.0,4267.0,131217.0,-5.01,46.0,X-SC,5.95,89.0,0.16,0.92,23.64,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.14,-14.87,40.12,19.28,118158.0,-51445.0,294511.0,-25.89,57.0,X-LC,7.02,1.0,-0.44,2.06,8.88,X40,ATH,IT
41,INFY,2275.00,-0.03,6.18,45.86,54.87,152354.0,19328.0,332216.0,-17.44,60.0,X-LC,2.29,2.0,0.13,2.33,14.97,X40,BTT,IT
76,TMPV,600.00,1.19,-14.44,66.18,42.18,156508.0,-39922.0,236488.0,-23.08,33.0,X-LC,2.32,3.0,-0.26,1.66,2.44,XY24,NTT,AUTO
82,VBL,671.64,0.09,-2.64,39.34,35.67,120977.0,-8326.0,307516.0,-14.21,62.0,X-LC,5.49,4.0,-0.07,2.15,10.85,X40N,ATH,FMCG
43,ITC,452.00,-0.19,-1.41,12.02,10.44,23717.0,-2826.0,197312.0,-41.15,45.0,X-LC,1.32,5.0,-0.12,1.38,4.07,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.38,-14.10,111.49,81.67,89845.0,-13224.0,80586.0,7244.44,47.0,L-SC,19.38,271.0,-0.15,0.56,56.06,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.58,-11.98,58.54,39.54,52124.0,-12119.0,89040.0,-31.16,65.0,M-SC,6.69,220.0,-0.23,0.62,25.56,AR,ATH,AUTO
51,MASFIN,398.61,1.44,-4.09,27.25,22.05,25608.0,-4005.0,93975.0,-17.34,54.0,H-SC,6.87,164.0,-0.16,0.66,36.60,XR,ATH,FINANCE
70,SURYODAY,216.00,1.42,-18.36,50.71,23.03,74131.0,-32885.0,146186.0,60.09,51.0,H-SC,9.72,167.0,-0.44,1.02,45.05,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.32,-8.53,9.34,0.02,14914.0,-14884.0,159675.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.89,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.32,-8.53,9.34,0.02,14914.0,-14884.0,159675.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.89,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,-0.75,-0.28,21.29,20.95,53450.0,-713.0,251055.0,-1.34,69.0,X-LC,15.74,31.0,-0.01,1.76,35.25,X40,ATH,PAINTS
50,LTIM,7201.88,0.33,11.82,17.75,31.66,48855.0,29091.0,275242.0,7.45,67.0,H-LC,13.95,43.0,0.60,1.93,52.72,X200,ATH,IT
0,5PAISA,593.00,-0.61,-36.62,78.13,12.90,108199.0,-80010.0,138486.0,104.33,53.0,H-SC,12.83,161.0,-0.74,0.97,15.15,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,-1.13,-37.33,112.06,32.89,54217.0,-28824.0,48382.0,-697.80,60.0,L-MC,5.81,259.0,-0.53,0.34,33.24,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.96
1,25,44.86
2,50,75.72


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.50
MC    29.09
LC    25.41
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.76
X40      21.22
X40N     12.02
XR        9.61
XY25      9.10
AR        9.03
OX40N     7.60
X200      1.93
SR        0.98
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.63
X-LC    20.14
M-SC    11.87
X-SC     8.31
H-MC     4.74
H-LC     3.08
L-SC     1.39
M-MC     1.38
M-LC     1.21
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.88,-4.33,38.07
IT,13.80,-14.66,73.85
FINANCE,9.48,-14.48,62.85
MISC,7.42,-22.60,72.83
ELECTRICAL,5.99,-9.69,49.87
PAINTS,5.94,-8.91,25.09
INSURANCE,4.39,-0.72,36.22
PHARMA,3.90,-0.47,32.62
AUTO,3.40,-18.25,68.96


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3069644.0,21
XR,1289519.0,13
AR,1282607.0,10
X40,1006172.0,14
X40N,730454.0,9
OX40N,705689.0,10
XY25,341324.0,6
SR,280944.0,2
MH,76902.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3560387.0,25
M-SC,1361969.0,15
X-MC,1248351.0,16
X-LC,885084.0,11
X-SC,733764.0,8
H-MC,401081.0,3
L-SC,262211.0,3
H-LC,118585.0,2
M-LC,118585.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1217566.0      6
           AR         881435.0      5
M-SC       XY24       784724.0      6
H-SC       XR         777058.0      7
X-MC       X40        444532.0      7
X-LC       X40        394446.0      5
X-MC       XY24       372112.0      3
H-SC       OX40N      326482.0      4
M-SC       OX40N      291667.0      5
X-SC       X40N       288965.0      3
H-SC       SR         280944.0      2
X-SC       XY24       277605.0      3
X-MC       X40N       249945.0      4
X-LC       XY24       227408.0      2
H-MC       AR         210852.0      2
X-LC       X40N       191544.0      2
H-MC       XY24       190229.0      1
X-MC       XY25       181762.0      2
L-SC       XR         174671.0      2
X-SC       X40        167194.0      2
M-SC       XR         164988.0      2
           AR         120590.0      2
M-LC       XR         118585.0      1
L-SC       OX40N       87540.0      1
H-SC       MH          76902.0      1
X-LC       XY25        71686.0      2
H-LC       AR          69730.0      1
L-MC       XR          54217.0      1
H-LC       X200        48855.0      1
M-MC       XY25        47829.0      1
L-LC       XY25        40047.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
